#Proyecto Introduccion a la IA

Entrenamiento de la base de datos de Hotel booking demand

###Universidad de Antioquia
###Facultad de Ingeniería
###Ingeniería de Sistemas
###UdeA - Ude@
###Profesor: Raul Ramos Pollan



### Integrantes

#### Nombre: Ramiro Monroy Ramos

#### Cédula: 1027949562



## Preparación y limpieza de datos. 
La base de datos utilizada esta compuesta por dos conjuntos de datos; el primero es de el hotel resort (HR)y otro es un hotel urbano(HU). 

##Instalación e importación de las librerias necesarias

In [5]:

# Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn import svm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import time
from mlxtend.feature_selection import SequentialFeatureSelector as SFS


In [7]:
# Cargar la base de datos
db = pd.read_csv('hotel_bookings.csv')
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [8]:
#Se limpia country por fuera del pipeline para que no quede agregado en el dataset final como categórico
db['country'].fillna(db['country'].mode()[0], inplace=True) 
# Se eliminan las columnas que no tienen sentido para el entrenamiento
# La característica se elimina porque solo existen 6797 valores de el total (119390) lo que corresponde a un porcentaje de faltante del 94.3%
# La característica reservation_status se elimina porque es una variable de salida
# La característica reservation_status_date se elimina porque es una fecha
db = db.drop(columns=['reservation_status', 'reservation_status_date','company'])
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [10]:
#Preparación del pipeline para entrenamiento

months= ['January','February','March', 'April', 'May', 'June'
        ,'July', 'August', 'September', 'October', 'November', 'December']


def procesar_con_onehotencoder(db):
    # Definir los transformadores
    #Imputer llena los valores nulos de las características children y agent con un valor 0. Son pocos los valores faltantes.
    #Para la característica agent falta un 13% de los datos. Para la característica children falta un 0.003% de los datos.
    #OrdinalEncoder le da un valor ordinal a los meses del año que se encuentran en la columna arrival_date_month.
    #OneHotEncoder convierte las variables categóricas a numéricas. No hay un orden específico.
    transformers = [
        ('imputer', SimpleImputer(strategy='constant', fill_value=0), ['children', 'agent'])
        ,('ordinal', OrdinalEncoder(categories=[months]), ['arrival_date_month'])
        ,('onehot', OneHotEncoder(sparse_output = False), ['hotel','country', 'meal', 'market_segment', 
                                                           'distribution_channel','reserved_room_type',
                                                           'assigned_room_type','deposit_type','customer_type'])
    ]

    # Crear el ColumnTransformer
    preprocessor = ColumnTransformer(transformers, remainder='passthrough')

    # Modelo para entrenamiento
    #svc = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

    # Crear el pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

    # Aplicar el pipeline al DataFrame
    transformed_data = pipeline.fit_transform(db)

    #transformed_data.shape

    # Convertir los resultados en un nuevo DataFrame
    #transformed_df = pd.DataFrame(transformed_data)
    return transformed_data

def procesar_con_ordinalencoder(db):
    # Definir los transformadores
    #Imputer llena los valores nulos de las características children y agent con un 0. Son pocos los valores faltantes.
    #Para la característica agent falta un 13% de los datos. Para la característica children falta un 0.003% de los datos.
    #OrdinalEncoder le da un valor ordinal a los meses del año que se encuentran en la columna arrival_date_month.
    #LabelEncoder convierte las variables categóricas a numéricas. No hay un orden específico.
    transformers = [
        ('imputer', SimpleImputer(strategy='constant', fill_value=0), ['children', 'agent'])
        ,('ordinal', OrdinalEncoder(categories=[months]), ['arrival_date_month'])
        ,('labelencoder', OrdinalEncoder(), ['hotel','country', 'meal', 'market_segment', 
                                                           'distribution_channel','reserved_room_type',
                                                           'assigned_room_type','deposit_type','customer_type'])
    ]

    # Crear el ColumnTransformer
    preprocessor = ColumnTransformer(transformers, remainder='passthrough')

    # Modelo para entrenamiento
    #svc = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

    # Crear el pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

    # Aplicar el pipeline al DataFrame
    transformed_data = pipeline.fit_transform(db)

    # retornar los resultados
    return transformed_data

def procesar_con_ordinalencoder_estandarizado(db):
    # Definir los transformadores
    #Imputer llena los valores nulos de las características children y agent con un 0. Son pocos los valores faltantes.
    #Para la característica agent falta un 13% de los datos. Para la característica children falta un 0.003% de los datos.
    #OrdinalEncoder le da un valor ordinal a los meses del año que se encuentran en la columna arrival_date_month.
    #LabelEncoder convierte las variables categóricas a numéricas. No hay un orden específico.
    transformers = [
        ('imputer', SimpleImputer(strategy='constant', fill_value=0), ['children', 'agent'])
        ,('ordinal', OrdinalEncoder(categories=[months]), ['arrival_date_month'])
        ,('labelencoder', OrdinalEncoder(), ['hotel','country', 'meal', 'market_segment', 
                                                           'distribution_channel','reserved_room_type',
                                                           'assigned_room_type','deposit_type','customer_type'])
    ]

    # Crear el ColumnTransformer
    preprocessor = ColumnTransformer(transformers, remainder='passthrough')

    # Modelo para entrenamiento
    #svc = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

    # Crear el pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

    # Aplicar el pipeline al DataFrame
    transformed_data = pipeline.fit_transform(db)
    
    #Aplicar estandarización
    scaler = MinMaxScaler()
    
    standardized_data = scaler.fit_transform(transformed_data)
    # retornar los resultados
    return standardized_data

In [11]:
datosX = db.drop('is_canceled', axis=1)

Y = np.asarray(db['is_canceled'])
X = procesar_con_ordinalencoder_estandarizado(datosX)
print(X.shape)
print(Y.shape)


(119390, 28)
(119390,)


In [6]:
X[0:5]

array([[0.        , 0.        , 0.54545455, 1.        , 0.76704545,
        0.        , 0.42857143, 0.25      , 0.22222222, 0.18181818,
        0.        , 0.66666667, 0.46404342, 0.        , 0.5       ,
        0.        , 0.        , 0.        , 0.03636364, 0.        ,
        0.        , 0.        , 0.        , 0.14285714, 0.        ,
        0.00118009, 0.        , 0.        ],
       [0.        , 0.        , 0.54545455, 1.        , 0.76704545,
        0.        , 0.42857143, 0.25      , 0.22222222, 0.18181818,
        0.        , 0.66666667, 1.        , 0.        , 0.5       ,
        0.        , 0.        , 0.        , 0.03636364, 0.        ,
        0.        , 0.        , 0.        , 0.19047619, 0.        ,
        0.00118009, 0.        , 0.        ],
       [0.        , 0.        , 0.54545455, 1.        , 0.33522727,
        0.        , 0.42857143, 0.25      , 0.        , 0.18181818,
        0.        , 0.66666667, 0.00949796, 0.        , 0.5       ,
        0.        , 0.    

In [15]:
###Funciones genéricas###

def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=0,
           scoring='accuracy',
           cv=0)
    
    return sfs

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)


def entrenar_con_kfold(clf, X, Y):
    errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    #Entrenar el modelo
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo sin hacer selección de características

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        errores[j] = classification_error(ypred, y_test)
        j+=1
    
    return errores
    
def entrenar_con_stratifiedkfold(clf, X, Y):
    errores = np.ones(10)
    j = 0

    skf = StratifiedKFold(n_splits=10)

    #Entrenar el modelo
    for i, (train_index, test_index) in enumerate(skf.split(X, Y)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entran y se valida el modelo sin hacer selección de características
        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        errores[j] = classification_error(ypred, y_test)
        j+=1
    
    return errores

def entrenar_con_seleccion_kfold(modelo, n_features, fwd, fltg):
    errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    #Entrenar el modelo
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo sin hacer selección de características

        # Entrenamiento el modelo.
        sf = select_features(modelo,
                        n_features,
                        fwd,
                        fltg)

        # Validación del modelo
        sf = sf.fit(X_train, y_train)

        #######

        errores[j] = 1-sf.k_score_
        j+=1
    
    return errores, sf


def entrenar_con_seleccion_stratifiedkfold(modelo, n_features, fwd, fltg):
    errores = np.ones(10)
    j = 0
    skf = StratifiedKFold(n_splits=10)

    #Entrenar el modelo
    for i, (train_index, test_index) in enumerate(skf.split(X, Y)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo sin hacer selección de características

        # Entrenamiento el modelo.
        sf = select_features(modelo,
                        n_features,
                        fwd,
                        fltg)

        # Validación del modelo
        sf = sf.fit(X_train, y_train)

        #######

        errores[j] = 1-sf.k_score_
        j+=1
    
    return errores, sf

def metrics(y_test, y_pred):
  '''
  Parametros:
    - lista (array) y_test (One-hot encoding)
    - lista (array) y_predichas (softmax vector)
  Return: 
    - F1
    - Recall
    - Precision
    - Accuracy
    - Especificity
  '''
  y_test = np.asarray(y_test)
  y_pred = np.asarray(y_pred)

  if y_test[0].shape[0] == 1:    #Biclase
    TP,TN,FP,FN = 0,0,0,0
    for t,p in zip(y_test,y_pred):
      if t==1:
        if p>=0.5: #yp==1:
          TP+=1
        else:
          FN+=1
      else:
        if p>=0.5: #yp==1:
          FP+=1
        else:
          TN+=1
    
    recall = TP/(TP+FN)
    prec = TP/(TP+FP)
    acc = (TP+TN)/(TP+FP+FN+TN)
    f1 = (2*recall*prec)/(recall+prec)
    esp = TN/(TN+FP)

    return f1, recall, prec, acc, esp
  
  else:    #Multiclase
    error = 0
    for test,pred in zip(y_test,y_pred):
        t = np.argmax(test)
        p = np.argmax(pred)
        if t!=p:
          error+=1
          
    err = error/np.shape(y_test)[0]     
    acc = 1 - err

    return acc